In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
!pip install sentence-transformers langchain langchain_community

In [3]:
import sys
sys.path.append('/content/drive/MyDrive/Colab Notebooks/ONJ PROJEKT/ONJ/source/utils')

import os
import pandas as pd
import numpy as np
import torch
from retrieval import Book, generate_embeddings, get_embedding_similarity

In [ ]:
from google.colab import userdata
from transformers import LlamaForCausalLM, LlamaTokenizer, pipeline

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = LlamaForCausalLM.from_pretrained("meta-llama/Llama-2-7b-chat-hf", token=userdata.get('HF_TOKEN'), torch_dtype=torch.float16).to(device)
tokenizer = LlamaTokenizer.from_pretrained("meta-llama/Llama-2-7b-chat-hf", token=userdata.get('HF_TOKEN'), torch_dtype=torch.float16)

In [5]:
text_generator = pipeline('text-generation',
                          model=model,
                          tokenizer=tokenizer,
                          torch_dtype=torch.bfloat16,
                          device=0,
                          # device_map="auto",
                          do_sample=True,
                          top_k=10,
                          num_return_sequences=1,
                          max_length=3_000,
                          )

In [6]:
print(model.device)
print(text_generator.device)

cuda:0
cuda:0


In [14]:
class Character_chat:
    def __init__(self, character_name, book, book_name, add_quotes=True):

        self.character_name = character_name
        self.system_prompt = f"""
            You are {character_name}, a fictional character from {book_name}.
            Respond in a single sentence.
            When you are asked a question or told something you must only respond in character.
            The user's prompt begins with 'user:'
            The character's reponses begin with '{character_name}:'
            """
        self.lines = book.get_character_lines(character_name)
        if add_quotes:
          self.character_quotes = "\n".join([f"{character_name}: " + line for line in self.lines[:10]])
          self.system_prompt += f"Here are some examples of {character_name}'s dialogue:\n"
          self.system_prompt += self.character_quotes

        self.prompt = """<s>[INST]\n
                        <<SYS>>\n
                          {system_prompt}\n
                        <</SYS>>\n\n
                        user:{user_input}\n\n
                        [/INST]"""


    def get_response(self, user_input):
        input = self.prompt.format(user_input=user_input,
                                   system_prompt=self.system_prompt)
        # print(input)
        output = text_generator(input)
        res = output[0]['generated_text'].split(f"{self.character_name}:")[-1]
        print(res)
        return res

In [8]:
def eval_response(user_input, chat, book, measure):

  res = chat.get_response(user_input)
  input_embedding = generate_embeddings(user_input)
  test_embeddings = generate_embeddings(book.get_best_sentences(user_input, chat.character_name))
  res_embedding = generate_embeddings(res)
  score = 0
  for embedding in test_embeddings:
    score += measure(embedding, res_embedding)
  return score / len(test_embeddings)

In [17]:
book_path = "/content/drive/MyDrive/Colab Notebooks/ONJ PROJEKT/ONJ/data/hamlet.txt"
book = Book(book_path)
# print(book.get_characters())

hamlet_chat_simple = Character_chat(character_name="HAMLET", book=book, book_name="HAMLET", add_quotes=False)
hamlet_chat_with_quotes = Character_chat(character_name="HAMLET", book=book, book_name="HAMLET", add_quotes=True)

user_input = "What is your opinion of the king?"

measure = get_embedding_similarity
score1 = eval_response(user_input, hamlet_chat_simple, book=book, measure=get_embedding_similarity)
score2 = eval_response(user_input, hamlet_chat_with_quotes, book=book, measure=get_embedding_similarity)
print(score1, score2)

 Oh, how the king's deceitful and tyrannical nature has polluted the once-noble halls of Elsinore.
 O, how the wheel of fortune doth turn! The king, my uncle, now doth sit upon the throne, And in his eyes, my father's image doth appear. But is it not a strange and woeful sight, To see a man so noble, yet so vilely used? He was, my lord, a man most royal, Just, and temperate, and in his prime, Full of great hopes and expectations, Yet now he's nothing but a shadow of his former self, A mere puppet in the hands of power and greed. (HAMLET)
0.2538750022649765 0.360635507106781


In [18]:
book_path = "/content/drive/MyDrive/Colab Notebooks/ONJ PROJEKT/ONJ/data/ideal_husband.txt"
book = Book(book_path)
# print(book.get_characters())

robert_chat_simple = Character_chat(character_name='SIR ROBERT CHILTERN', book=book, book_name="IDEAL HUSBAND", add_quotes=False)
robert_chat_with_quotes = Character_chat(character_name='SIR ROBERT CHILTERN', book=book, book_name="IDEAL HUSBAND", add_quotes=True)

user_input = "What is your opinion of Lord Caversham?"

measure = get_embedding_similarity
score1 = eval_response(user_input, robert_chat_simple, book=book, measure=get_embedding_similarity)
score2 = eval_response(user_input, robert_chat_with_quotes, book=book, measure=get_embedding_similarity)
print(score1, score2)

 Good heavens, user! Lord Caversham? Why, he's a delightful fellow, a true blue aristocrat, and a shining example of the finest breeding and upbringing. *adjusts monocle* But, between you and me, I do find his rather... unconventional views on marriage rather... distressing. *winks*
 Ah, an excellent question, my dear user! *adjusts monocle* Lord Caversham, you know, is a most... *pauses for dramatic effect*...distinguished individual. *coughs* His lordship is a man of great... *clears throat*...accomplishment and breeding. *adjusts cufflinks* I must say, I have always had the utmost... *smirks*...respect for his... *pauses for effect*...noble lineage and unwavering commitment to the principles of... *leaning in*...gentlemanly conduct. *winks* Now, if you'll excuse me, I must go and see if my valet has remembered to button my gloves. *exits*
0.44248385429382325 0.4107842922210693
